In [1]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '8' encode 6
defense '10' encode 7


In [5]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
from tqdm import tqdm, tqdm_notebook
from joblib import Parallel, delayed
import warnings

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out, indices_out = [], [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]
        indices_c = np.where(y == label)[0]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        # Ensure at least one sample per class
        n_sample = max(1, n_sample)

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])
        indices_out.append(indices_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0), np.concatenate(indices_out, axis=0)

def compute_entropy(prob):
    return entropy(prob, base=2)

def eoal_sampling(X_pool, y_pool, model, fraction, known_class, diversity=True, random_state=42, use_notebook=False):

    np.random.seed(random_state)
    
    # Standardize features for better clustering
    scaler = StandardScaler()
    X_pool_scaled = scaler.fit_transform(X_pool)
    
    # Get model predictions and probabilities
    softprobs = model.predict_proba(X_pool)
    y_pred = model.predict(X_pool)
    
    # Parallel compute entropy for large datasets
    entropy_list = np.array(Parallel(n_jobs=-1)(delayed(compute_entropy)(prob) for prob in softprobs))
    
    # Identify known and unknown samples
    known_mask = y_pred < known_class
    queryIndex = np.arange(len(X_pool))
    queryIndex_k = queryIndex[known_mask]
    uncertaintyArr_k = entropy_list[known_mask]
    labelArr_k = y_pool[known_mask]
    
    # Calculate number of samples to select
    query_batch = min(int(fraction * len(X_pool)), len(queryIndex_k))  # Ensure query_batch <= len(queryIndex_k)
    
    if not diversity:
        # Uncertainty-based sampling only
        sorted_idx = np.argsort(uncertaintyArr_k)[-query_batch:]  # Select top uncertain samples
        selected_idx = queryIndex_k[sorted_idx]
        selected_gt = labelArr_k[sorted_idx]
    else:
        # Diversity-based sampling with MiniBatchKMeans clustering
        embeddings_k = X_pool_scaled[known_mask]
        # Estimate number of clusters
        num_clusters = min(known_class, query_batch, len(embeddings_k))
        if num_clusters < 1:
            num_clusters = 1
        kmeans = MiniBatchKMeans(n_clusters=num_clusters, random_state=random_state, batch_size=1000)
        cluster_labels = kmeans.fit_predict(embeddings_k)
        
        # Allocate budget per cluster
        rem = min(query_batch, len(queryIndex_k))
        num_per_cluster = max(1, rem // num_clusters)  # Ensure at least 1 sample per cluster
        selected_idx = []
        selected_gt = []
        ax = [0] * num_clusters
        
        # Initialize tqdm progress bar
        tqdm_func = tqdm_notebook if use_notebook else tqdm
        with tqdm_func(total=rem, desc="Sampling Progress", unit="samples") as pbar:
            while rem > 0:
                for cls in range(num_clusters):
                    temp_ent = uncertaintyArr_k[cluster_labels == cls]
                    temp_index = queryIndex_k[cluster_labels == cls]
                    temp_gt = labelArr_k[cluster_labels == cls]
                    # Limit samples to remaining budget
                    samples_to_take = min(num_per_cluster, len(temp_ent), rem)
                    if samples_to_take > 0:
                        sorted_idx = np.argsort(temp_ent)[-samples_to_take:]
                        ax[cls] += len(sorted_idx)
                        rem -= len(sorted_idx)
                        selected_idx.extend(temp_index[sorted_idx])
                        selected_gt.extend(temp_gt[sorted_idx])
                        pbar.update(len(sorted_idx))  # Update progress bar
                pbar.set_description(f"Sampling Progress (Remaining: {rem})")
                if rem <= 0:
                    break  # Exit loop if budget exhausted
        
        selected_idx = np.array(selected_idx)
        selected_gt = np.array(selected_gt)
    
    # Handle remaining budget with unknown samples
    if len(selected_gt) < query_batch:
        rem_budget = query_batch - len(selected_idx)
        uncertaintyArr_u = entropy_list[~known_mask]
        queryIndex_u = queryIndex[~known_mask]
        labelArr_u = y_pool[~known_mask]
        
        sorted_idx_extra = np.argsort(uncertaintyArr_u)[-rem_budget:]
        selected_idx = np.concatenate((selected_idx, queryIndex_u[sorted_idx_extra]))
        selected_gt = np.concatenate((selected_gt, labelArr_u[sorted_idx_extra]))
    
    # Calculate precision and recall
    Len_labeled_ind_train = len(np.where(y_pool < known_class)[0])  # Assume all known samples in pool
    precision = len(np.where(selected_gt < known_class)[0]) / len(selected_gt) if len(selected_gt) > 0 else 0
    recall = len(np.where(selected_gt < known_class)[0]) / (
        len(np.where(y_pool < known_class)[0]) + Len_labeled_ind_train) if (
        len(np.where(y_pool < known_class)[0]) + Len_labeled_ind_train) > 0 else 0
    
    return selected_idx, selected_gt, precision, recall

def batch_mode_active_learning(X_pool, y_pool, model, fraction=0.5, random_state=42, use_notebook=False):

    known_class = len(np.unique(y_pool))  # Assume all classes in y_pool are known for simplicity
    selected_idx, selected_gt, _, _ = eoal_sampling(
        X_pool, y_pool, model, fraction, known_class, diversity=True, random_state=random_state, use_notebook=use_notebook
    )
    
    X_selected = X_pool[selected_idx]
    y_selected = selected_gt
    remaining_mask = np.ones(len(X_pool), dtype=bool)
    remaining_mask[selected_idx] = False
    X_remaining = X_pool[remaining_mask]
    y_remaining = y_pool[remaining_mask]
    
    return X_remaining, X_selected, y_remaining,y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init, _ = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)

# Perform batch mode active learning with fractions 50%, 20%, 1%
_, X_20, _, y_20 = batch_mode_active_learning(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.2, random_state=42)

Sampling Progress (Remaining: 0): 100%|██████████| 286503/286503 [00:00<00:00, 953993.45samples/s]     


In [6]:
X_20.shape

(286503, 42)

In [7]:
np.unique(y_20, return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([188059,     10,  51435,      3,     20,  46970,      1,      5]))

In [8]:
from xgboost import XGBClassifier

print('xgb')
xgb20 = XGBClassifier()
xgb20.fit(X_20, y_20)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf20 = RandomForestClassifier()
rf20.fit(X_20, y_20)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt20 = DecisionTreeClassifier()
# dt20.fit(X_20, y_20)

xgb
RF


RandomForestClassifier()

In [9]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [10]:
import numpy as np



base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb20.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_EOAL_Exclude3Attack/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 5 6] [     9      1      1 238742]
(9, 42) (9,)
Save 2 to baseline_Def2.npy
(1, 42) (1,)
Save 3 to baseline_Def3.npy
(1, 42) (1,)
Save 5 to baseline_Def5.npy
(238742, 42) (238742,)
Save 6 to baseline_Def6.npy
Execution Time: 0.388697 seconds
(716259,)
[1 3 4 5 6] [508295  26396     99     10 181459]
(508295, 42) (508295,)
Save 1 to BIM_Def1.npy
(26396, 42) (26396,)
Save 3 to BIM_Def3.npy
(99, 42) (99,)
Save 4 to BIM_Def4.npy
(10, 42) (10,)
Save 5 to BIM_Def5.npy
(181459, 42) (181459,)
Save 6 to BIM_Def6.npy
Execution Time: 1.185808 seconds
(716259,)
[ 1  2  3  4  5  6 10] [501593     42    966    411      3 213150     94]
(501593, 42) (501593,)
Save 1 to FGSM_Def1.npy
(42, 42) (42,)
Save 2 to FGSM_Def2.npy
(966, 42) (966,)
Save 3 to FGSM_Def3.npy
(411, 42) (411,)
Save 4 to FGSM_Def4.npy
(3, 42) (3,)
Save 5 to FGSM_Def5.npy
(213150, 42) (213150,)
Save 6 to FGSM_Def6.npy
(94, 42) (94,)
Save 10 to FGSM_Def10.npy
Execution Time: 0.835655 seconds
(716259,)
[1 3 4 5 6] [508295

In [11]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf20.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_EOAL_Exclude3Attack/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 3 5 6] [     6      1      1 238745]
(6, 42) (6,)
Save 2 to baseline_Def2.npy
(1, 42) (1,)
Save 3 to baseline_Def3.npy
(1, 42) (1,)
Save 5 to baseline_Def5.npy
(238745, 42) (238745,)
Save 6 to baseline_Def6.npy
Execution Time: 0.849479 seconds
(716259,)
[1 2 3 6] [454725 223211  23094  15229]
(454725, 42) (454725,)
Save 1 to BIM_Def1.npy
(223211, 42) (223211,)
Save 2 to BIM_Def2.npy
(23094, 42) (23094,)
Save 3 to BIM_Def3.npy
(15229, 42) (15229,)
Save 6 to BIM_Def6.npy
Execution Time: 4.476879 seconds
(716259,)
[1 2 3 6] [477932  38447    195 199685]
(477932, 42) (477932,)
Save 1 to FGSM_Def1.npy
(38447, 42) (38447,)
Save 2 to FGSM_Def2.npy
(195, 42) (195,)
Save 3 to FGSM_Def3.npy
(199685, 42) (199685,)
Save 6 to FGSM_Def6.npy
Execution Time: 3.568024 seconds
(716259,)
[1 2 3 6] [454725 223211  23094  15229]
(454725, 42) (454725,)
Save 1 to PGD_Def1.npy
(223211, 42) (223211,)
Save 2 to PGD_Def2.npy
(23094, 42) (23094,)
Save 3 to PGD_Def3.npy
(15229, 42) (15229,)
Save 6 to 

In [12]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt20.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_EOAL_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ActiveLearning_EOAL_Exclude3Attack/WUSTL_Input20/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [13]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}20")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ActiveLearning_EOAL_Exclude3Attack/WUSTL_Input20/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [14]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input20_ActiveLearning_EOAL_Exclude3Attack.csv")

In [15]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,0,9,1,0,1,238742,0,0,0,0,0,238753
1,XGB20,508295,0,26396,99,10,181459,0,0,0,0,0,716259
2,XGB20,501593,42,966,411,3,213150,0,0,0,94,0,716259
3,XGB20,508295,0,26396,99,10,181459,0,0,0,0,0,716259
4,XGB20,702130,2,13907,0,80,28,0,0,0,112,0,716259
5,XGB20,417654,6,71501,8,184,226899,0,0,0,7,0,716259
6,XGB20,0,31,1,0,7,716220,0,0,0,0,0,716259
7,XGB20,212730,8537,24087,0,151,470754,0,0,0,0,0,716259
8,XGB20,22479,0,469932,9,0,223839,0,0,0,0,0,716259
9,XGB20,546516,0,5584,508,2,163631,0,0,0,18,0,716259
